In [3]:
import requests
import pandas as pd
from io import StringIO
import csv
from tqdm import tqdm_notebook as tqdm
import sqlite3


In [4]:
#grabbing finance data from yahoo finance for use:

company_list = ['AMZN', 'TSLA', 'AAPL']
stockdata = []
def getData(company_stock):
    url = f'https://query1.finance.yahoo.com/v7/finance/download/{company_stock}?'
    params = {
           'range': '1y',
           'interval':'1mo',
           'events':'history',
           'includeAdjustedClose':'true'}
    url_info = requests.get(url,params = params, headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})
    #print(url_info.text)
    file = StringIO(url_info.text)
    reader = csv.reader(file)
    data = list(reader)
    #print(data)

    data_frame = pd.DataFrame(data[1:], columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
    data_frame.insert(1, 'Company Name', f'{company_stock}')
    return pd.DataFrame(data_frame)
    
for company in company_list:
    stockdata.append(getData(company))
    print(f'Getting: {company}')


Getting: AMZN
Getting: TSLA
Getting: AAPL


In [5]:
combined_list = pd.concat(stockdata)
print(combined_list)


          Date Company Name        Open        High         Low       Close  \
0   2022-02-01         AMZN  150.000000  163.834503  138.332993  153.563004   
1   2022-03-01         AMZN  152.732498  170.831497  133.572495  162.997498   
2   2022-04-01         AMZN  164.149506  168.394501  121.625000  124.281502   
3   2022-05-01         AMZN  122.401001  126.220497  101.260002  120.209503   
4   2022-06-01         AMZN  122.255997  128.990005  101.430000  106.209999   
5   2022-07-01         AMZN  106.290001  137.649994  105.849998  134.949997   
6   2022-08-01         AMZN  134.960007  146.570007  126.739998  126.769997   
7   2022-09-01         AMZN  126.000000  136.490005  112.059998  113.000000   
8   2022-10-01         AMZN  113.580002  123.000000   97.660004  102.440002   
9   2022-11-01         AMZN  103.989998  104.580002   85.870003   96.540001   
10  2022-12-01         AMZN   96.989998   97.230003   81.690002   84.000000   
11  2023-01-01         AMZN   85.459999   98.370003 

In [6]:
#Using SQLite3: 
connection = sqlite3.connect('practice_finance.sqlite3') #Creating local database file
cursor = connection.cursor()
cursor.execute('Drop Table if exists stock_data')
#Creating table:
tables_1 = 'Create Table stock_data (id integer, date datetime, company_name varchar(100) , open integer, high integer, low integer, close integer, adj_close integer, volume integer)'  
cursor.execute(tables_1)

connection.commit()

In [7]:
#inserting information:
insert_amazon = 'Insert into stock_data (id, date, company_name, open, high, low, close, adj_close, volume) values (?, ?, ?, ?, ?, ?, ?, ?, ?)'
cursor = connection.cursor()
for info in combined_list.itertuples():
    cursor.execute(insert_amazon, info)
connection.commit()


In [8]:
#Query:
query_1 = ''' 
select * from stock_data where company_name = 'AMZN'
'''
for i in cursor.execute(query_1):
    print(i)

(0, '2022-02-01', 'AMZN', 150, 163.834503, 138.332993, 153.563004, 153.563004, 1689604000)
(1, '2022-03-01', 'AMZN', 152.732498, 170.831497, 133.572495, 162.997498, 162.997498, 1628486000)
(2, '2022-04-01', 'AMZN', 164.149506, 168.394501, 121.625, 124.281502, 124.281502, 1465008000)
(3, '2022-05-01', 'AMZN', 122.401001, 126.220497, 101.260002, 120.209503, 120.209503, 2258476000)
(4, '2022-06-01', 'AMZN', 122.255997, 128.990005, 101.43, 106.209999, 106.209999, 1767601100)
(5, '2022-07-01', 'AMZN', 106.290001, 137.649994, 105.849998, 134.949997, 134.949997, 1337852600)
(6, '2022-08-01', 'AMZN', 134.960007, 146.570007, 126.739998, 126.769997, 126.769997, 1170449000)
(7, '2022-09-01', 'AMZN', 126, 136.490005, 112.059998, 113, 113, 1210487600)
(8, '2022-10-01', 'AMZN', 113.580002, 123, 97.660004, 102.440002, 102.440002, 1459311500)
(9, '2022-11-01', 'AMZN', 103.989998, 104.580002, 85.870003, 96.540001, 96.540001, 2035133200)
(10, '2022-12-01', 'AMZN', 96.989998, 97.230003, 81.690002, 84, 84

In [9]:
query_2 = ''' 
select date, company_name, max(close), min(close) from stock_data
'''
for i in cursor.execute(query_2):
    print(i)

('2022-12-01', 'AMZN', 359.200012, 84)


In [10]:
amzn_data_maximum = ''' 
select date, avg(open), avg(close), max(close), min(close) from stock_data where company_name = 'AMZN'
'''
for i in cursor.execute(amzn_data_maximum):
    print(i)




('2022-12-01', 120.99915438461538, 117.0155006923077, 162.997498, 84)


In [ ]:
#using python to reverse the process as in opening a database file and saving it as csv

sql_query = pd.read_sql_query(amzn_data_maximum, connection)
data_file_python = pd.DataFrame(sql_query)
print(data_file_python)

connection.close()

In [28]:
#saving the python file for later

data_file_python.to_csv('Amazon_stock_info.csv')